# [World Bank API](https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-about-this-api-documentation)


### API Queries: Argument vs URL

- **Argument-based** query strings require you to pass in *arguments* to the URL to inform it of what data you're looking for. This is the type of query string that we've been working with in class today.
    - Example: [`http://api.worldbank.org/V2/countries?incomeLevel=LIC`](http://api.worldbank.org/V2/countries?incomeLevel=LIC)
- **URL-based** query strings are pre-configured URL paths that direct you to a canned response.
    - Example: [`http://api.worldbank.org/V2/incomeLevels/LIC/countries`](http://api.worldbank.org/V2/incomeLevels/LIC/countries)

# 1. Request all countries and associated data from API

#### Import dependencies and construct variables

In [102]:
import requests
from pprint import pprint

# variables
url = "http://api.worldbank.org/v2/"

# default response format = XML
response_format = "json"

# default number of countries per page = 50
countries_per_page = 4

#### Compile the query URL

**NOTE:** We technically could have just created one long string with all of our arguments in the string, but using variables to contain those arguments makes our code easier to read and it allows us to quickly adjust argument values via the respective variables.

In [103]:
# Compile the query URL
query_url = f"{url}countries?format={response_format}&per_page={countries_per_page}"
print(query_url)

http://api.worldbank.org/v2/countries?format=json&per_page=4


#### Execute the GET request

In [104]:
countries_response = requests.get(query_url).json()

#### Explore the response

Since every API returns the response in a different format, we could first take a quick look at the response type and length.

In [99]:
print(f"Response Type: {type(countries_response)}")
print(f"Response Length: {len(countries_response)}")

Response Type: <class 'list'>
Response Length: 2


#### Since our response has 2 items in the list, we should take a look at each of them.

##### The first item provides a general summary of the response that includes the total number of countries that meet our selection.

##### The second item provides the actual countries and their associated data

In [106]:
print("RESPONSE ITEM 1:")
print("-"*25)
pprint(countries_response[0])

print()

print("RESPONSE ITEM 2:")
print("-"*25)
pprint(countries_response[1])

RESPONSE ITEM 1:
-------------------------
{'page': 1, 'pages': 76, 'per_page': '4', 'total': 304}

RESPONSE ITEM 2:
-------------------------
[{'adminregion': {'id': '', 'iso2code': '', 'value': ''},
  'capitalCity': 'Oranjestad',
  'id': 'ABW',
  'incomeLevel': {'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
  'iso2Code': 'AW',
  'latitude': '12.5167',
  'lendingType': {'id': 'LNX', 'iso2code': 'XX', 'value': 'Not classified'},
  'longitude': '-70.0167',
  'name': 'Aruba',
  'region': {'id': 'LCN',
             'iso2code': 'ZJ',
             'value': 'Latin America & Caribbean '}},
 {'adminregion': {'id': 'SAS', 'iso2code': '8S', 'value': 'South Asia'},
  'capitalCity': 'Kabul',
  'id': 'AFG',
  'incomeLevel': {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'},
  'iso2Code': 'AF',
  'latitude': '34.5228',
  'lendingType': {'id': 'IDX', 'iso2code': 'XI', 'value': 'IDA'},
  'longitude': '69.1761',
  'name': 'Afghanistan',
  'region': {'id': 'SAS', 'iso2code': '8S', 'value'

#### Print the country names

In [107]:
countries_list = countries_response[1]

for country in countries_list:
    print(country["name"])

Aruba
Afghanistan
Africa
Angola


# 2. Make a more selective request for income level by country

#### API Request for South Asian countries

In [171]:
# region selection for South Asia
region = "SAS"

# Compile query URL using the more selective argument variables
selective_query_url = f"{url}countries?format={response_format}&region={region}"

# Execute GET request
selective_countries_response = requests.get(selective_query_url).json()

# Print the URL for clarity
print(selective_query_url)

http://api.worldbank.org/v2/countries?format=json&region=SAS


#### Explore the response

In [175]:
# How many SAS countries are there?
print("Selective Count: ", selective_countries_response[0]['total'])
print()

# Print each country and the respective income level
for c in selective_countries_response[1]:
    print(f"Country: {c['name']}")
    print(f"Income: {c['incomeLevel']['id']} - {c['incomeLevel']['value']}")
    print()

Selective Count:  8

Country: Afghanistan
Income: LIC - Low income

Country: Bangladesh
Income: LMC - Lower middle income

Country: Bhutan
Income: LMC - Lower middle income

Country: India
Income: LMC - Lower middle income

Country: Sri Lanka
Income: LMC - Lower middle income

Country: Maldives
Income: UMC - Upper middle income

Country: Nepal
Income: LIC - Low income

Country: Pakistan
Income: LMC - Lower middle income



## 3. What if we want to find out how many countries fall into each income level

#### Using a different API offered by the World Bank, we can first obtain just the unique income levels

In [194]:
# compile URL for strictly the income levels
income_url = f"{url}incomelevel?format={response_format}"

# Execute GET request and convert to json
income_response = requests.get(tmp_url).json()

pprint(income_response[1])

[{'id': 'HIC', 'iso2code': 'XD', 'value': 'High income'},
 {'id': 'INX', 'iso2code': 'XY', 'value': 'Not classified'},
 {'id': 'LIC', 'iso2code': 'XM', 'value': 'Low income'},
 {'id': 'LMC', 'iso2code': 'XN', 'value': 'Lower middle income'},
 {'id': 'LMY', 'iso2code': 'XO', 'value': 'Low & middle income'},
 {'id': 'MIC', 'iso2code': 'XP', 'value': 'Middle income'},
 {'id': 'UMC', 'iso2code': 'XT', 'value': 'Upper middle income'}]


#### Create a list of all incomeLevel IDs

In [195]:
incomes = [income['id'] for income in income_response[1]]
print(incomes)

['HIC', 'INX', 'LIC', 'LMC', 'LMY', 'MIC', 'UMC']


#### Loop through the income values and execute a GET request for countries that are associated with each income value

In [212]:
# create empty dictionary to add to within the loop
inc_dict = {}

# loop over all unique incomes
for i in incomes:
    
    # Execute a GET request for all countries FILTERED BY incomeLevel
    income_country_response = requests.get(f"{url}countries?format={response_format}&incomeLevel={i}").json()
    
    # add an entry to the empty dictionary where the key is the incomeLevel and the value is the total number of countries that are associated with that incomeLevel
    inc_dict[i] = income_country_response[0]['total']
    
# print the dictionary counts    
print(inc_dict)

{'HIC': 81, 'INX': 0, 'LIC': 34, 'LMC': 47, 'LMY': 137, 'MIC': 103, 'UMC': 56}
